In [1]:
import pandas as pd
from tqdm import tqdm
import networkx as nx
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import transformers

In [2]:
df = pd.read_pickle("metadata_full_filtered.pkl")

In [3]:
model = AutoModelForCausalLM.from_pretrained("bhavinjawade/SOLAR-10B-OrcaDPO-Jawade", device_map="auto", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained("bhavinjawade/SOLAR-10B-OrcaDPO-Jawade")

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [4]:
df = df[df["high_confidence_concept"]].reset_index(drop=True).iloc[0:10]

In [5]:
df

,doi,id,abstract,title,publication_date,type,type_crossref,keywords,first_author,all_authors,all_authors_af,language,referenced_works,referenced_works_count,related_works,concepts,is_accepted,is_published,high_confidence_concept
0,https://doi.org/10.1103/physrevlett.86.4536,https://openalex.org/W1973501299,"the instability, dynamics, and morphological t...",templating of thin films induced by dewetting ...,2001-05-14,article,journal-article,"['thin films', 'surfaces']",Kajari Kargupta,"['Kajari Kargupta', 'Ashutosh Sharma']","['Department of Chemical Engineering, Indian I...",en,"['https://openalex.org/W1514074674', 'https://...",23,"['https://openalex.org/W2236704256', 'https://...","[('Dewetting', 0.96603024), ('Instability', 0....",False,False,True
1,https://doi.org/10.1103/physrevd.49.5194,https://openalex.org/W2141784194,the dilaton free energy density in external st...,finite temperature dilaton gravity,1994-05-15,article,journal-article,"['gravity', 'temperature', 'finite']",Kopeck Z,['Kopeck Z'],['Department of Theoretical Physics and Astrop...,en,"['https://openalex.org/W1964629348', 'https://...",14,"['https://openalex.org/W3183872805', 'https://...","[('Dilaton', 0.95246434), ('Physics', 0.872703...",False,False,True
2,https://doi.org/10.1103/physrevlett.86.103,https://openalex.org/W2019376583,we report on a strong overtone delocalization ...,two-phonon states and overtone delocalization ...,2001-01-01,article,journal-article,"['vibrational spectroscopy', 'oscillators', 'o...",T. Moritz,"['T. Moritz', 'W. Widdra']","['Physik Department E20, Technische Universita...",en,"['https://openalex.org/W1510223905', 'https://...",10,"['https://openalex.org/W2043651069', 'https://...","[('Overtone', 0.97254896), ('Anharmonicity', 0...",False,False,True
3,https://doi.org/10.1103/physrevb.86.100501,https://openalex.org/W2068631041,we report the observation of a pseudogap in th...,pseudogap in underdoped ba1−xkxfe2as<…,2012-09-04,article,journal-article,['ba<mmlmath xmlnsmml=http//wwww3org/1998/math...,Y. M. Dai,"['Y. M. Dai', 'B. Xu', 'B. Shen', 'H. H. Wen',...",['Beijing National Laboratory for Condensed Ma...,en,"['https://openalex.org/W1481943103', 'https://...",36,"['https://openalex.org/W1988781771', 'https://...","[('Pseudogap', 0.98302376), ('Physics', 0.6606...",False,False,True
4,https://doi.org/10.1103/physrevb.100.174409,https://openalex.org/W2971444323,we investigated the self-induced inverse spin ...,self-induced inverse spin hall effect in ferro...,2019-11-06,article,journal-article,"['inverse spin hall effect', 'ferromagnets', '...",O. Gladii,"['O. Gladii', 'L. Frangou', 'A. Hallal', 'R. S...","['Univ. Grenoble Alpes, CNRS, CEA, Grenoble IN...",en,"['https://openalex.org/W1554201985', 'https://...",42,"['https://openalex.org/W2999737531', 'https://...","[('Permalloy', 0.96676517), ('Condensed matter...",False,False,True
5,https://doi.org/10.1103/physrevd.47.4244,https://openalex.org/W2080096802,a model of baryogenesis is described which giv...,baryon isocurvature fluctuations at small scal...,1993-05-15,article,journal-article,"['baryon isocurvature fluctuations', 'baryonic...",Alexandre Dolgov,"['Alexandre Dolgov', 'Joseph Silk']","['Center for Particle Astrophysics, University...",en,"['https://openalex.org/W1965240959', 'https://...",34,"['https://openalex.org/W1909431038', 'https://...","[('Physics', 0.95786417), ('Baryogenesis', 0.7...",False,False,True
6,https://doi.org/10.1103/physrevd.59.043512,https://openalex.org/W2066304100,we study a double inflation model (a preinflat...,primordial black hole formation in supergravity,1999-01-25,article,journal-article,"['primordial black hole formation', 'black hole']",M. Kawasaki,"['M. Kawasaki', 'T. Yanagida']",['Institute for Cosmic Ray Research University...,en,"['https://openalex.org/W1689540754', 'https://...",34,"['https://openalex.org/W2030979397', 'https://...","[('Inflaton', 0.9604207), ('Physics', 0.915529...",False,False,True
7,https://doi.org/10.1103/phy

In [6]:
from random import random

def get_queries_args_for_llm_task(df):
    args = []
    for _, row in df.iterrows():
        args.append({
            "concepts_random": sorted([m[0] for m in eval(row["concepts"])], key=lambda z: random()),
            "concepts_open_alex": [m[0] for m in eval(row["concepts"])],
            "abstract": row["abstract"],
            "title": row["title"]
        })
    return args
    

In [7]:
queries_args = get_queries_args_for_llm_task(df)

In [8]:
TASK = f"""
You are a ranking system for sorting a list of scientific concepts related to a paper abstract.

Given a paper title, abstract and a random list of scientific concepts, you will sort the list by the most related to the least related to the abstract. 
As input, you will receive queries in the following format:

QUERY_START
TITLE_START
{queries_args[0]["title"]}
TITLE_END
ABSTRACT_START
{queries_args[0]["abstract"]}
ABSTRACT_END
CONCEPTS_START
{str(queries_args[0]["concepts_random"])}
CONCEPTS_END
QUERY_END

In a correct response you are expected to yield a list of, AT MAXIMUM 10, scientific concepts sorted from the most related to the least related to the abstract. 
Failure in following this format will imply in harsh punishments.

As output it is expected the following response, given the example query above:

RESPONSE_START
[(1, 'Dewetting'),
 (2, 'Instability'),
 (3, 'Breakup'),
 (4, 'Substrate (aquarium)'),
 (5, 'Materials science'),
 (6, 'Pattern formation'),
 (7, 'Thin film'),
 (8, 'Lambda'),
 (9, 'Length scale'),
 (10, 'Condensed matter physics')]
RESPONSE_END
"""

In [16]:
TASK = f"""
You are a ranking system for sorting a list of scientific concepts related to a paper abstract.

Given a paper title and abstract, you will create a list of scientific concepts and sort it the a by the most related to the least related to the abstract and title. 
As input, you will receive queries in the following format:

QUERY_START
TITLE_START
{queries_args[0]["title"]}
TITLE_END
ABSTRACT_START
{queries_args[0]["abstract"]}
ABSTRACT_END
QUERY_END

In a correct response you are expected to yield a list of, AT MAXIMUM 10, scientific concepts sorted from the most related to the least related to the abstract. 
Failure in following this format will imply in harsh punishments.

As output it is expected the following response, given the example query above:

RESPONSE_START
[(1, 'Dewetting'),
 (2, 'Instability'),
 (3, 'Breakup'),
 (4, 'Substrate (aquarium)'),
 (5, 'Materials science'),
 (6, 'Pattern formation'),
 (7, 'Thin film'),
 (8, 'Lambda'),
 (9, 'Length scale'),
 (10, 'Condensed matter physics')]
RESPONSE_END
"""

In [9]:
QUERY_TEMPLATE = """
QUERY_START
TITLE_START
{0}
TITLE_END
ABSTRACT_START
{1}
ABSTRACT_END
CONCEPTS_START
{2}
CONCEPTS_END
QUERY_END
"""

In [17]:
QUERY_TEMPLATE = """
QUERY_START
TITLE_START
{0}
TITLE_END
ABSTRACT_START
{1}
ABSTRACT_END
QUERY_END
"""

In [10]:
print(QUERY_TEMPLATE.format(queries_args[0]["title"], queries_args[0]["abstract"], str(queries_args[0]["concepts_random"])))


QUERY_START
TITLE_START
templating of thin films induced by dewetting on patterned surfaces
TITLE_END
ABSTRACT_START
the instability, dynamics, and morphological transitions of patterns in thin liquid films on chemically heterogeneous striped surfaces are investigated based on 3d nonlinear simulations. the film breakup is suppressed on some potentially destabilizing nonwettable sites when their spacing is below a characteristic length scale of the instability, lambda(h). the thin film pattern replicates the substrate surface energy pattern closely only when (i) the periodicity of substrate pattern lies between lambda(h) and 2lambda(h), and (ii) the stripe width is within a range bounded by a lower critical length, below which no heterogeneous rupture occurs, and an upper transition length above which complex morphological features unlike the substrate pattern are formed.
ABSTRACT_END
CONCEPTS_START
['Thin film', 'Physics', 'Materials science', 'Condensed matter physics', 'Biology', 'B

In [14]:
print()


RESPONSE_START
[(1, 'Dilaton'),
 (2, 'Gravitation'),
 (3, 'Gravitational field'),
 (4, 'Einstein'),
 (5, 'Finite temperature'),
 (6, 'String (physics)'),
 (7, 'Thermal'),
 (8, 'Thermodynamics'),
 (9, 'Einstein gravity'),
 (10, 'Theoretical physics')]
RESPONSE_END


In [18]:
responses = []
for q in queries_args[1:]:
    #query = QUERY_TEMPLATE.format(q["title"], q["abstract"], str(q["concepts_random"]))
    query = QUERY_TEMPLATE.format(q["title"], q["abstract"])
    message = [
        {"role": "system", "content": TASK},
        {"role": "user", "content": query}
    ]
    prompt = tokenizer.apply_chat_template(message, add_generation_prompt=True, tokenize=False)
    pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer
    )
    sequences = pipeline(
        prompt,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        num_return_sequences=1,
        max_length=1500,
    )
    r = sequences[0]['generated_text'].split("### Assistant:")[1]
    responses.append(r)
    print(r, q["concepts_open_alex"])


RESPONSE_START
[(1, 'Dilaton Gravity'),
 (2, 'Finite Temperature'),
 (3, 'Free Energy Density'),
 (4, 'Static Gravitational Field'),
 (5, 'Real Time Formulation'),
 (6, 'Finite Temperature Field Theory'),
 (7, 'String Parameter ($\alpha '$)),
 (8, 'Thermal Corrections'),
 (9, 'Modified Einstein Gravity'),
 (10, 'Condensed Matter Physics/Theoretical Physics')]
RESPONSE_END ['Dilaton', 'Physics', 'Gravitation', 'Energy density', 'Gravitational field', 'Einstein', 'Thermal', 'String (physics)', 'Quantum electrodynamics', 'Classical mechanics', 'Gravitational energy', 'Mathematical physics', 'Theoretical physics', 'Thermodynamics']

RESPONSE_START
[(1, 'Two-phonon states'),
 (2, 'Overtone delocalization'),
 (3, 'One-dimensional systems'),
 (4, 'Oscillators'),
 (5, 'Vibrational spectroscopy'),
 (6, 'No adsorbate row structure'),
 (7, 'Phonon dispersion'),
 (8, 'Electron energy loss spectroscopy'),
 (9, 'Adsorbate systems'),
 (10, 'Ru(001) surface')]
RESPONSE_END ['Overtone', 'Anharmonicity